In [0]:
import pandas as pd
import numpy as np
from scipy import sparse

Some tips to work with Colab: https://dev.to/kriyeng/8-tips-for-google-colab-notebooks-to-take-advantage-of-their-free-of-charge-12gb-ram-gpu-be4

We wil work with the [ASSISTments2012](https://sites.google.com/site/assistmentsdata/home/2012-13-school-data-with-affect) dataset.
The link is provided for documentation because the data are already available from the notebook.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!cp "/gdrive/My Drive/Colab Notebooks/data/assistments12/data.zip" .

In [0]:
df = pd.read_csv("data.zip", compression='zip')

In [5]:
df.head()

,problem_log_id,skill,problem_id,user_id,assignment_id,assistment_id,start_time,end_time,problem_type,original,correct,bottom_hint,hint_count,actions,attempt_count,ms_first_response,tutor_mode,sequence_id,student_class_id,position,type,base_sequence_id,skill_id,teacher_id,school_id,overlap_time,template_id,answer_id,answer_text,first_action,problemlogid,Average_confidence(FRUSTRATED),Average_confidence(CONFUSED),Average_confidence(CONCENTRATING),Average_confidence(BORED)
0,137792159,NaN,557460,61394,565736,341511,2012-09-28 15:11:27,2012-09-28 15:11:36.856,choose_1,1,1.0,0.0,0,"--- \n- - start\n - 1348859487561\n - ""95952...",1,9852,tutor,55482,23643,4,LinearSection,55482,NaN,53472,5048.0,9852,341511,NaN,she,0,137792159,0.361323,0.0,0.336529,0.000000
1,138083797,Rounding,365981,61394,573819,204043,2012-10-09 11:01:52,2012-10-09 11:02:13.182,algebra,1,1.0,0.0,0,"--- \n- - start\n - 1349794912269\n - ""62459...",1,21175,tutor,34221,22967,5,LinearSection,34221,54.0,47424,5048.0,21175,204043,NaN,74.29,0,138083797,0.361323,0.0,0.766925,0.000000
2,142332619,Multiplication and Division Integers,426415,61394,734130,247525,2013-03-07 10:53:20,2013-03-07 10:53:28.661,algebra,1,0.0,0.0,0,"--- \n- - start\n - 1362671600405\n - ""74107...",1,8645,tutor,39601,22967,58,LinearSection,39601,279.0,47424,5048.0,8645,247525,NaN,00,0,142332619,0.361323,0.0,0.766925,0.442968
3,145939397,Proportion,86686,61394,821352,48081,2013-08-20 19:54:56,2013-08-20 19:55:21.753,algebra,1,1.0,0.0,0,"--- \n- - start\n - 1377042896503\n - ""73630...",1,25728,tutor,6912,26303,21,MasterySection,6912,79.0,47424,5048.0,25728,46362,NaN,3.8,0,145939397,0.775000,0.0,0.766925,0.912281
4,137111284,NaN,399669,76592,557216,227869,2012-09-10 17:20:10,2012-09-10 17:24:56.579,choose_1,1,1.0,0.0,0,"--- \n- - start\n - 1347312010563\n - ""69479...",1,286578,tutor,37143,21696,3,LinearSection,37143,NaN,152676,7561.0,286578,227869,NaN,C (wr - 1)(wr + 1),0,137111284,0.361323,0.0,0.766925,0.000000


Way too much information !
Try to remove some.
We will keep user_di, item_id, timestamps, correct and inter_id (index)

In [0]:
min_interactions_per_user = 10
remove_nan_skills = True

df["timestamp"] = df["start_time"]
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["timestamp"] = df["timestamp"] - df["timestamp"].min()
#df["timestamp"] = df["timestamp"].apply(lambda x: x.total_seconds() / (3600*24))
df["timestamp"] = df["timestamp"].apply(lambda x: x.total_seconds()).astype(np.int64)
df.sort_values(by="timestamp", inplace=True)
df.reset_index(inplace=True, drop=True)
df = df.groupby("user_id").filter(lambda x: len(x) >= min_interactions_per_user)

if remove_nan_skills:
  df = df[~df["skill_id"].isnull()]
else:
  df.ix[df["skill_id"].isnull(), "skill_id"] = -1

df["user_id"] = np.unique(df["user_id"], return_inverse=True)[1]
df["item_id"] = np.unique(df["problem_id"], return_inverse=True)[1]
df["skill_id"] = np.unique(df["skill_id"], return_inverse=True)[1]



In [0]:
# Build Q-matrix
Q_mat = np.zeros((len(df["item_id"].unique()), len(df["skill_id"].unique())))
item_skill = np.array(df[["item_id", "skill_id"]])
for i in range(len(item_skill)):
  Q_mat[item_skill[i,0],item_skill[i,1]] = 1

In [0]:
df.reset_index(inplace=True, drop=True) # Add unique identifier of the row
df["inter_id"] = df.index
df = df[['user_id', 'item_id', 'timestamp', 'correct', "inter_id"]]
df = df[df.correct.isin([0,1])] # Remove potential continuous outcomes
df['correct'] = df['correct'].astype(np.int32) # Cast outcome as int32


In [12]:
df.head()

,user_id,item_id,timestamp,correct,inter_id
0,4355,27076,0,0,0
1,3825,23707,85,1,1
2,3825,23680,103,0,2
3,4355,26976,122,1,3
4,3825,23625,190,0,4


In [13]:
len(df)

2692889

Save the dataframe and the Q-matrix

In [0]:
# Save data
sparse.save_npz("q_mat.npz", sparse.csr_matrix(Q_mat))
df.to_csv("preprocessed_data.csv", sep="\t", index=False)

Copy the data to GDrive

In [0]:
!cp q_mat.npz "/gdrive/My Drive/Colab Notebooks/data/assistments12/"
!cp preprocessed_data.csv "/gdrive/My Drive/Colab Notebooks/data/assistments12/"